In [27]:
import pandas as pd
import numpy as np
import itertools
from itertools import compress
import time
import os



Load the data

- procurement_arr : the procurement centres [id, weight, name]
- transplant_arr : the transplant centres [name,weight]
- possible_arr : for each transplant centres, the procurement centres ids to be combined. Order is the same as transplant_arr

In [28]:
procurement_arr = np.load('./data/procurment_list.pkl.npy',allow_pickle=True)
transplant_arr = np.load('./data/transplant_list.pkl.npy',allow_pickle=True)
possible_arr = np.load('./data/possible_list.pkl.npy',allow_pickle=True)

A dictionary to replace procurement centres ids with weight

In [29]:
procdict = dict(
    zip(procurement_arr[:,0], procurement_arr[:,1]))
procdict['-1'] = 0

## 1. Calculating all unique combinations for each transplant centres

The combine function

In [30]:
def combi(_combi_list, _w_transplant, _min_proc, _max_proc, _min_ratio, _max_ratio, _transdict):
    """Creating unique procurement site combinations for a transplant team

    :param _combi_list: the list of procurement sites associated with a team
    :param _w_transplant: Transplant centre weight
    :param _min_proc: the minimum number of procurement centre combinations to be associated with the transplant team
    :param _max_proc: the maximum number of procurement centre combinations to be associated with the transplant team
    :param _min_ratio: the minimum ratio transplant weight / procurement weight to be retained for the transplant team
    :param _max_ratio: the maximum ratio transplant weight / procurement weight to be retained for the transplant team
    :param _transdict: procurement centres dict with ids and weights
    :return: two nparrays of unique combinations of procurements centers and ratio for the transplant team
    """

    def _ratio(ar, dic, n):
        """Calculate combination's ratio after replacing the ids of the procurements centres with their weight"""

        k = np.array(list(dic.keys()))
        v = np.array(list(dic.values()))
        sidx = k.argsort()
        ks = k[sidx]
        vs = v[sidx]
        sort = vs[np.searchsorted(ks, ar)]
        ratio = np.sum(sort, axis=1) / n
        ratio = np.round(ratio, decimals=7)
        return ratio

    _max_proc = _max_proc if len(_combi_list) > _max_proc else len(_combi_list)
    print(f'The maximum number of procurement centres after filtering is : {_max_proc}')

    all_combinations = np.empty((0, _max_proc))
    all_ratios = np.empty((0), 'f16')
    all_n = np.empty((0), int)

    for r in range(_min_proc, _max_proc + 1):
        start_time = time.time()
        l_shape = abs(r - _max_proc)

        combinations_object = itertools.combinations(_combi_list, r)
        combinations_np = np.array(list(combinations_object))
        combinations_np = np.pad(combinations_np, ((0, 0), (0, l_shape)), mode='constant', constant_values=-1)
        ratio_np = _ratio(combinations_np, _transdict, _w_transplant)

        # removing rows below _min_ratio and above _max_ratio
        masked_combinations = combinations_np[
            np.where(np.logical_and(ratio_np >= _min_ratio, ratio_np <= _max_ratio), True, False)]
        masked_ratio = ratio_np[np.where(np.logical_and(ratio_np >= _min_ratio, ratio_np <= _max_ratio), True, False)]

        all_combinations = np.append(all_combinations, masked_combinations, axis=0)
        all_ratios = np.append(all_ratios, masked_ratio, axis=0)
        print(f"{r} --- %s seconds --- {time.time() - start_time}")
    return all_combinations, all_ratios

The solve function

In [50]:
def solve(_transplant_arr,_possible_arr, t_idx, transdict, min_proc=2, max_proc=6, min_ratio=0.4, max_ratio=1):
    """Generating all possible combinations according to the constraints

    :param _transplant_arr: the transplant centres array
    :param possible_arr: for each transplant centres, the procurement centres ids to be combined. Order is the same as _transplant_arr
    :param t_idx: transplant center index
    :param transdict: the dictionary to replace procurement centres ids with weight
    :param min_proc: the minimum number of procurement centre combinations to be associated with the transplant team
    :param max_proc: the maximum number of procurement centre combinations to be associated with the transplant team
    :param min_ratio: the minimum ratio transplant weight / procurement weight to be retained for the transplant team
    :param max_ratio: the maximum ratio transplant weight / procurement weight to be retained for the transplant team
    :return: two nparrays of unique combinations of procurements centers and ratio for the transplant team
    """

    print(f"Combinations for: {_transplant_arr[t_idx][0]}")

    all_comb, all_ratio = combi(_combi_list=_possible_arr[t_idx],
                                _w_transplant=_transplant_arr[t_idx][1],
                                _min_proc=min_proc,
                                _max_proc=max_proc,
                                _min_ratio=min_ratio,
                                _max_ratio=max_ratio,
                                _transdict=transdict)

    mask_count = np.where(all_comb > 0, True, False)
    all_n = mask_count.sum(axis=1)

    df = pd.concat([pd.DataFrame(all_ratio, columns={'ratio'}),
                    pd.DataFrame(all_n.astype(int), columns={'n'}),
                    pd.DataFrame(all_comb.astype(int))],

                   axis=1, sort=False)

    new_names = [(i, 'proc_id_' + str(i)) for i in df.iloc[:, 2:].columns.values]
    df.rename(columns=dict(new_names), inplace=True)

    df = df.sort_values(by=['ratio', 'n'], ascending=[False, True])
    df = df.astype({'ratio': 'float64'})
    df.reset_index(drop=True, inplace=True)
    path = './results'
    
    isExist = os.path.exists(path)
    os.makedirs(path) if not isExist else print(f"Path {path} already exists")
    df.to_pickle(f"./results/{_transplant_arr[t_idx][0]}.pckl")
    print("--- end ---")

Solve for transplant_arr idx = 0 (E13MARS004215) and idx = 1 (E31TOUL052555)

In [58]:
solve(_transplant_arr=transplant_arr,
      _possible_arr=possible_arr,
      t_idx = 0, #E13MARS004215
      transdict=procdict,
      min_proc=5,
      max_proc=11,
      min_ratio=0.4,
      max_ratio=0.6)

solve(_transplant_arr=transplant_arr,
      _possible_arr=possible_arr,
      t_idx = 1, #E31TOUL052555
      transdict=procdict,
      min_proc=5,
      max_proc=11,
      min_ratio=0.4,
      max_ratio=0.6)

Combinations for: E13MARS004215
The maximum number of procurement centres after filtering is : 11
5 --- %s seconds --- 0.030103206634521484
6 --- %s seconds --- 0.05016064643859863
7 --- %s seconds --- 0.04602527618408203
8 --- %s seconds --- 0.03326606750488281
9 --- %s seconds --- 0.029117822647094727
10 --- %s seconds --- 0.013484716415405273
11 --- %s seconds --- 0.0047607421875
Path ./results already exists
--- end ---
Combinations for: E31TOUL052555
The maximum number of procurement centres after filtering is : 11
5 --- %s seconds --- 0.008717536926269531
6 --- %s seconds --- 0.008169174194335938
7 --- %s seconds --- 0.0062351226806640625
8 --- %s seconds --- 0.003612041473388672
9 --- %s seconds --- 0.0013575553894042969
10 --- %s seconds --- 0.001178741455078125
11 --- %s seconds --- 0.0008060932159423828
Path ./results already exists
--- end ---


Dataframes of each combination with the procurement weight / transplant weight ratio and procurment centres ids

E13MARS004215

In [90]:
solve_eqp0 = pd.read_pickle(f"./results/{transplant_arr[0][0]}.pckl")
solve_eqp0.head()

,ratio,n,proc_id_0,proc_id_1,proc_id_2,proc_id_3,proc_id_4,proc_id_5,proc_id_6,proc_id_7,proc_id_8,proc_id_9,proc_id_10
0,0.6,5,18,144,143,142,41,-1,-1,-1,-1,-1,-1
1,0.6,5,18,144,143,41,7,-1,-1,-1,-1,-1,-1
2,0.6,5,18,144,143,41,6,-1,-1,-1,-1,-1,-1
3,0.6,6,17,20,144,143,41,33,-1,-1,-1,-1,-1
4,0.6,6,18,20,144,143,142,5,-1,-1,-1,-1,-1


E31TOUL052555

In [91]:
solve_eqp1 = pd.read_pickle(f"./results/{transplant_arr[1][0]}.pckl")
solve_eqp1.head()

,ratio,n,proc_id_0,proc_id_1,proc_id_2,proc_id_3,proc_id_4,proc_id_5,proc_id_6,proc_id_7,proc_id_8,proc_id_9,proc_id_10
0,0.59434,5,42,43,12,140,44,-1,-1,-1,-1,-1,-1
1,0.59434,5,42,43,12,14,102,-1,-1,-1,-1,-1,-1
2,0.59434,5,42,43,140,44,69,-1,-1,-1,-1,-1,-1
3,0.59434,5,42,43,140,44,48,-1,-1,-1,-1,-1,-1
4,0.59434,5,42,43,14,69,102,-1,-1,-1,-1,-1,-1
